In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
data = requests.get('https://api.covid19india.org/v4/data-all.json').json()

for date in data.keys():
    date_dict = data[date]
    # Remove all the states which don't have district data in them
    date_dict = {state : state_dict for state, state_dict in date_dict.items() if 'districts' in state_dict.keys()}
    data[date] = date_dict
    
# Remove all the dates which have 0 states with district data after pruning
data = {date : date_dict for date, date_dict in data.items() if len(date_dict) > 0}

# Make the districts key data the only data available for the state key
for date in data.keys():
    for state in data[date].keys():
        # Make the districts key dict the main dict itself for a particular date, state
        data[date][state] = data[date][state]['districts']
        state_dict = data[date][state]
        # Keep only those district dicts for which cumulative data (total key) is available
        state_dict = {dist : dist_dict for dist, dist_dict in state_dict.items() if 'total' in dist_dict.keys()}
        data[date][state] = state_dict

        # Make the total key dict the main dict itself for a particular date, state, dist
        for district in data[date][state].keys():
                data[date][state][district] = data[date][state][district]['total']
        
        # For a particular date, state, dist, only keep those keys for which have confirmed, recovered, deceased are all available
        state_dict = {dist : dist_dict for dist, dist_dict in state_dict.items() if {'confirmed', 'recovered', 'deceased'} <= dist_dict.keys()}
        data[date][state] = state_dict
    
    # Remove all the states for a particular date which don't have district that satisfied above criteria
    date_dict = data[date]
    date_dict = {state : state_dict for state, state_dict in date_dict.items() if len(state_dict) > 0}
    data[date] = date_dict
    
# Remove all the dates which have 0 states with district data after pruning
data = {date : date_dict for date, date_dict in data.items() if len(date_dict) > 0}

In [ ]:
data['2020-04-29']

In [ ]:
df_districts_all = pd.DataFrame(columns=['date', 'state', 'district', 'confirmed', 'active', 'recovered', 'deceased', 'tested', 'migrated'])
for date in data.keys():
    for state in data[date].keys():
        df_date_state = pd.DataFrame.from_dict(data[date][state]).T.reset_index()
        df_date_state = df_date_state.rename({'index' : 'district'}, axis='columns')
        df_date_state['active'] = df_date_state['confirmed'] - (df_date_state['recovered'] + df_date_state['deceased'])
        df_date_state['state'] = statecode_to_state_dict[state]
        df_date_state['date'] = date
        df_districts_all = pd.concat([df_districts_all, df_date_state], ignore_index=True)

In [ ]:
df_mumbai = df_districts_all.loc[(df_districts_all['state'] == 'Maharashtra') & (df_districts_all['district'] == 'Mumbai'), :]

In [ ]:
numeric_cols = ['confirmed', 'active', 'recovered', 'deceased', 'tested', 'migrated']
df_mumbai.loc[:, numeric_cols] = df_mumbai.loc[:, numeric_cols].apply(pd.to_numeric)

In [ ]:
df_mumbai